## Imports and setup

In [ ]:
!pip install wandb pytorch_lightning torchmetrics polars catboost

In [ ]:
import os
import random
import sys
import collections
import importlib
from abc import ABC, abstractmethod

import wandb
import numpy as np
import pandas as pd
import polars as pls
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm import tqdm, trange

import sklearn
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error


import catboost
from catboost import Pool, CatBoostRegressor
from catboost import EShapCalcType, EFeaturesSelectionAlgorithm


import torch
import torchmetrics
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

from torchmetrics import R2Score, MeanSquaredError, MeanAbsolutePercentageError

import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers import WandbLogger

from collections import deque, defaultdict

In [ ]:
# optional
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
seed_everything(123456)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

## Data

In [ ]:
!rm -rf qber-forecasting
!rm -rf deep_qber
!git clone https://github.com/rmnigm/qber-forecasting.git
!cp -r qber-forecasting/deep_qber deep_qber

In [ ]:
path = "/content/qber-forecasting/datasets/qber_with_outliers.csv"
raw_dataframe = pd.read_csv(path)

info_path = "/content/qber-forecasting/datasets/outliers_info.csv"
info_dataframe = pd.read_csv(info_path)

In [ ]:
with_steps = (
    raw_dataframe
    .set_index('index')
    .join(info_dataframe.set_index('index')[['steps_to_anomaly']],
          on='index',
          how='left',
          rsuffix='_info'
          )
    )

with_steps['outliers'] = (with_steps['steps_to_anomaly'] == 0).astype(int)
wo_junk = with_steps.drop(columns='steps_to_anomaly')

In [ ]:
from deep_qber.dataset import TorchDatasetInterface

In [ ]:
window_size = 20
start = 20
end = len(raw_dataframe)
target_column = 'e_mu_current'
anomaly_column = 'outliers'

datavault = TorchDatasetInterface(wo_junk,
                                  target_column,
                                  anomaly_column='outliers',
                                  window_size=20,
                                  train_size=0.75
                                  )

In [ ]:
datavault.setup(20, end, from_files=False)
dataloaders = datavault.get_dataloaders(anomaly_split=True, device=device)

## Model

In [ ]:
config = {
    "learning_rate": 1e-5,
    "look_back": window_size,
    "hidden_size": 256,
    "output_size": 1,
    "input_size": 7,
    "batch_size": 256,
    "epochs": 10,
    "loss": "MSE",
    "scaler": None,
    "train_size": 0.75,
}

loss = nn.MSELoss()
scaler = None

In [ ]:
from deep_qber.model import ModuleTS, ModelInterfaceTS
from deep_qber.model import Extractor, ExtractorExod, ExtractorLSTM

In [ ]:
def run_experiment(train_loader, test_loader, model, loss, config, device):
    with wandb.init(project="qber_v2",
                    entity="rmnigm",
                    settings=wandb.Settings(start_method="thread"),
                    config=config,
                    ) as run:
        wandb_logger = WandbLogger(log_model='all')
        checkpoint_callback = ModelCheckpoint(monitor="Validation R2Score", mode="max")

        epochs = config["epochs"]

        model_interface = ModelInterfaceTS(model, device)
        module = ModuleTS(model_interface, loss, lr=config["learning_rate"])

        trainer = pl.Trainer(logger=wandb_logger,
                            callbacks=[checkpoint_callback],
                            accelerator="gpu",
                            max_epochs=epochs,
                            )

        trainer.fit(module, train_loader, test_loader)

        run.finish()

In [ ]:
model = ExtractorLSTM(
    look_back=config["look_back"],
    output_size=config["output_size"],
    input_size=config["input_size"],
    hidden_size=config["hidden_size"],
    )

In [ ]:
config["model"] = "lstm; normal data"
run_experiment(train_normal_loader, test_normal_loader, model, loss, config, device)

In [ ]:
config["model"] = "lstm; anomaly data"
run_experiment(train_anomaly_loader, test_anomaly_loader, model, loss, config, device)